# 🌊 Light Wave Photoelectric Effect Explorer

Interactive notebook for exploring the wave-based photoelectric model.

This model investigates whether photoelectric-like behavior can emerge from pure wave mechanics with an internal resonance mechanism.

## The Model

**State Variables:**
- x(t): Electron displacement
- v(t): Electron velocity  
- s(t): Internal store/gain

**Equations:**
```
dx/dt = v
dv/dt = -ω₀²x - d·v + s·u(t)
ds/dt = -α·s + β·f²·x·u(t)
```

The **f²** term creates a frequency threshold!


In [ ]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add project root to path
sys.path.insert(0, str(Path('.').absolute()))

from src.physics import AtomModel, EscapeDetector
from src.simulation import Simulation, SimulationConfig, create_default_simulation
from src.analysis import find_threshold_frequency, analyze_escape_times
from src.visualization import (
    plot_time_series, plot_phase_portrait, 
    plot_frequency_sweep, plot_energy_evolution
)
from config.parameters import (
    PhysicsParameters, DEFAULT_PHYSICS, 
    PHOTOELECTRIC_TUNED, estimate_threshold_frequency
)

# Set up nice plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("✓ All modules loaded successfully!")


## 1. Set Up Parameters and Run Single Simulations


In [ ]:
# TUNED parameters that show clear photoelectric threshold behavior
# With these parameters, threshold is around f ~ 0.12-0.15
params = PhysicsParameters(
    omega0=1.0,       # Natural binding frequency
    damping=0.0001,   # Very low damping - allows oscillation growth
    alpha=0.05,       # Low store decay rate
    beta=100.0,       # Strong frequency-dependent coupling
    amplitude=0.2,    # Moderate wave amplitude
    escape_position=5.0
)

print(params.describe())

# Estimate threshold analytically
est_thresh = estimate_threshold_frequency(params)
print(f"\nEstimated threshold frequency: {est_thresh:.3f}")


In [ ]:
# Create simulation with tuned parameters
sim = create_default_simulation(
    omega0=params.omega0, damping=params.damping,
    alpha=params.alpha, beta=params.beta,
    amplitude=params.amplitude,
    escape_threshold=params.escape_position,
    t_max=150.0, dt=0.0005  # Small timestep for accuracy
)

# Below threshold (should stay bound) - threshold is ~0.12
freq_below = 0.08
result_below = sim.run(freq_below)
print(f"f = {freq_below}: Escaped = {result_below.escaped}")

# Above threshold (should escape)
freq_above = 0.2
result_above = sim.run(freq_above)
if result_above.escaped:
    print(f"f = {freq_above}: Escaped = True, t = {result_above.escape_time:.2f}")
else:
    print(f"f = {freq_above}: Escaped = {result_above.escaped}")


In [ ]:
# Plot comparison: below vs above threshold
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Below threshold row
axes[0, 0].plot(result_below.times, result_below.positions, 'b-', alpha=0.7)
axes[0, 0].set_title(f'Position (f={freq_below}, BOUND)')
axes[0, 0].set_ylabel('x(t)')

axes[0, 1].plot(result_below.times, result_below.velocities, 'r-', alpha=0.7)
axes[0, 1].set_title('Velocity')
axes[0, 1].set_ylabel('v(t)')

axes[0, 2].plot(result_below.times, result_below.stores, 'orange', alpha=0.7)
axes[0, 2].set_title('Internal Store')
axes[0, 2].set_ylabel('s(t)')

# Above threshold row
axes[1, 0].plot(result_above.times, result_above.positions, 'b-', alpha=0.7)
axes[1, 0].set_title(f'Position (f={freq_above}, ESCAPED)')
axes[1, 0].set_ylabel('x(t)')
axes[1, 0].set_xlabel('Time')

axes[1, 1].plot(result_above.times, result_above.velocities, 'r-', alpha=0.7)
axes[1, 1].set_title('Velocity')
axes[1, 1].set_xlabel('Time')

axes[1, 2].plot(result_above.times, result_above.stores, 'orange', alpha=0.7)
axes[1, 2].set_title('Internal Store (grows → escape)')
axes[1, 2].set_xlabel('Time')

plt.suptitle('Below vs Above Threshold: Key Difference is Store Growth!', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()


## 2. Frequency Sweep: Finding the Threshold

Let's sweep through frequencies to find exactly where the threshold is.


In [ ]:
# Frequency sweep
frequencies = np.linspace(0.2, 2.5, 40)

results = []
print("Running frequency sweep...")
for i, f in enumerate(frequencies):
    result = sim.run(f)
    results.append(result)
    status = "ESCAPE" if result.escaped else "bound"
    if (i+1) % 10 == 0:
        print(f"  [{i+1}/{len(frequencies)}] f={f:.3f} → {status}")

# Find threshold
threshold = find_threshold_frequency(results)
print(f"\n{'='*50}")
print(f"THRESHOLD FREQUENCY: {threshold.threshold_frequency:.4f}")
print(f"Uncertainty: ±{threshold.threshold_uncertainty:.4f}")
print(f"{'='*50}")


In [ ]:
# Visualize frequency sweep
plot_frequency_sweep(results, threshold.threshold_frequency, show=True)


## 3. KEY TEST: Amplitude Independence

A crucial characteristic of the real photoelectric effect: the threshold frequency doesn't depend on light intensity (amplitude).

This is what Einstein's "photon" explanation was designed to explain. But can wave mechanics with internal resonance also explain it?


In [ ]:
# Test multiple amplitudes
amplitudes = [0.001, 0.005, 0.01, 0.02, 0.05]
frequencies = np.linspace(0.3, 2.5, 30)

results_by_amp = {}
thresholds_by_amp = {}

print("Testing amplitude independence...")
print("="*50)

for amp in amplitudes:
    sim_amp = create_default_simulation(
        omega0=params.omega0, damping=params.damping,
        alpha=params.alpha, beta=params.beta,
        amplitude=amp, escape_threshold=params.escape_position,
        t_max=200.0
    )
    
    results = [sim_amp.run(f) for f in frequencies]
    results_by_amp[amp] = results
    
    thresh = find_threshold_frequency(results)
    thresholds_by_amp[amp] = thresh.threshold_frequency
    
    print(f"Amplitude {amp:.4f} → Threshold {thresh.threshold_frequency:.4f}")

# Calculate variation
thresh_values = list(thresholds_by_amp.values())
thresh_mean = np.mean(thresh_values)
thresh_std = np.std(thresh_values)
variation = thresh_std / thresh_mean

print("="*50)
print(f"Mean threshold: {thresh_mean:.4f}")
print(f"Variation: {variation:.2%}")
print("="*50)

if variation < 0.1:
    print("✓ AMPLITUDE INDEPENDENT! (Photoelectric-like behavior)")
else:
    print("✗ Threshold varies with amplitude - adjust parameters")


In [ ]:
# Plot threshold vs amplitude
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Threshold vs amplitude (should be flat!)
ax1 = axes[0]
ax1.scatter(amplitudes, thresh_values, s=100, c='purple', edgecolor='white', linewidth=2)
ax1.axhline(thresh_mean, color='gray', linestyle='--', label=f'Mean = {thresh_mean:.3f}')
ax1.set_xlabel('Wave Amplitude A', fontsize=12)
ax1.set_ylabel('Threshold Frequency', fontsize=12)
ax1.set_title('Threshold vs Amplitude\n(Flat = Photoelectric behavior!)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Escape curves for each amplitude
ax2 = axes[1]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(amplitudes)))

for amp, color in zip(amplitudes, colors):
    freqs = [r.frequency for r in results_by_amp[amp]]
    escaped = [1 if r.escaped else 0 for r in results_by_amp[amp]]
    ax2.plot(freqs, escaped, 'o-', color=color, label=f'A={amp}', alpha=0.7, markersize=4)

ax2.axvline(thresh_mean, color='red', linestyle='--', linewidth=2, label='Threshold')
ax2.set_xlabel('Frequency', fontsize=12)
ax2.set_ylabel('Escaped (0/1)', fontsize=12)
ax2.set_title('All amplitudes have SAME threshold!', fontsize=12)
ax2.legend(loc='center right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Parameter Exploration

Try modifying parameters below to see how they affect threshold behavior!


In [ ]:
def explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=1.0):
    """
    Quick parameter exploration.
    
    Key relationships:
    - Threshold ~ sqrt(alpha/beta)
    - Higher omega0 = stronger binding
    - Lower damping = easier buildup
    """
    sim = create_default_simulation(
        omega0=omega0, damping=damping, alpha=alpha, beta=beta,
        amplitude=0.01, escape_threshold=5.0, t_max=200.0
    )
    
    frequencies = np.linspace(0.2, 3.0, 30)
    results = [sim.run(f) for f in frequencies]
    thresh = find_threshold_frequency(results)
    
    print(f"Parameters: ω₀={omega0}, d={damping}, α={alpha}, β={beta}")
    print(f"Analytical estimate: ~{np.sqrt(alpha/beta):.3f}")
    print(f"Actual threshold: {thresh.threshold_frequency:.3f}")
    
    # Quick plot
    fig, ax = plt.subplots(figsize=(10, 4))
    freqs = [r.frequency for r in results]
    escaped = [1 if r.escaped else 0 for r in results]
    colors = ['green' if not e else 'red' for e in escaped]
    ax.scatter(freqs, escaped, c=colors, s=80)
    ax.axvline(thresh.threshold_frequency, color='purple', linestyle='--', 
               label=f'Threshold = {thresh.threshold_frequency:.3f}')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Escaped')
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['No', 'Yes'])
    ax.legend()
    ax.set_title(f'ω₀={omega0}, d={damping}, α={alpha}, β={beta}')
    plt.show()

# Try default parameters
explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=1.0)


In [ ]:
# TRY THESE! Uncomment to explore different parameters:

# Higher alpha (faster store decay) -> higher threshold
# explore_parameters(omega0=1.0, damping=0.01, alpha=1.0, beta=1.0)

# Higher beta (stronger coupling) -> lower threshold  
# explore_parameters(omega0=1.0, damping=0.01, alpha=0.5, beta=2.0)

# Stronger binding
# explore_parameters(omega0=2.0, damping=0.01, alpha=0.5, beta=1.0)


## 5. Summary

### What This Model Demonstrates:

1. **Frequency threshold emerges** from the f² coupling term - no photons needed!
2. **Amplitude independence** naturally arises from the internal dynamics
3. **Internal store mechanism** explains frequency-selective response

### The Key Physics Insight:

The photoelectric threshold can emerge from:
- Frequency-dependent internal dynamics (the f² term)
- Resonance/gain mechanisms (the store variable s)
- Nonlinear feedback between wave driving and internal state

### What This Means:

Light behaving as a wave CAN explain the photoelectric effect if we model matter with appropriate internal structure. The "particle-like" behavior emerges from the **internal dynamics of the atom**, not from light itself being particles!

### Next Steps:
1. Add output wave emission (for Compton scattering)
2. Model different materials (different parameters)
3. Explore multi-frequency driving
